# Import Packages

In [ ]:
from gurobipy import *
import numpy as np
import pandas as pd

# Parameters

In [4]:
# Input
input_file_name = "queen5.5"
file_type = ".txt"
file_path = ""

In [5]:
timeLimit = 3600 * 24
output_file_path = ""

# Read Data

In [ ]:
data = open(file_path + input_file_name + file_type,'r')

data = data.read()
data=data.split('\n')
data=pd.DataFrame(data)

print("Data size: ",data.shape)
data.head()

In [ ]:
data=data[0].str.split(' ', expand=True)
data.head()

# Build Model

In [ ]:
n = int(data.iloc[0][0])
K = int(data.iloc[0][1])
p = int(data.iloc[0][2])

E = int(data.iloc[1][1])

data=data.iloc[:,:3]

# dropping first 2 rows
data=data.drop([0,1])

# dropping all rows with 'None'
data.drop(data[data.isnull().any(axis=1)].index,inplace=True)

print("Data Shape: ",data.shape)
data.head()

In [7]:
edge_matrix = np.zeros((n,n))

for e in range(0,data.shape[0]):
    i = int(data.iloc[e][1])
    j = int(data.iloc[e][2])
    
    if j>=i:
        edge_matrix[i-1][j-1] = 1
    else:
        edge_matrix[j-1][i-1] = 1

In [ ]:
# Create a new model
m1 = Model("MSCP1")

# Logging
m1.Params.LogFile = output_file_path + input_file_name + "_m1_24hrs.log"
m1.Params.LogToConsole = 0

# Create variables
X = m1.addVars(n, K, vtype=GRB.BINARY, name="X")

# Objective function
obj_func = LinExpr()

for i in range(1,n+1):
    for j in range(1,K+1):
        obj_func.addTerms(j,X[i-1,j-1])

# Constraints
# Constraint 1
for i in range(1,n+1):
    constraint_1 = LinExpr()
    for j in range(1,K+1):
        constraint_1.addTerms(1,X[i-1,j-1])
    m1.addConstr(constraint_1 == 1)

# Constraint 2        
for i in range(0,n):
    for j in range(i,n):
        if (edge_matrix[i][j] == 1):
            for k in range(0,K):
                m1.addConstr(X[i,k] + X[j,k] <= 1)


# Set objective
m1.setObjective(obj_func, GRB.MINIMIZE)

# Time limit
m1.Params.timelimit = timeLimit

# Optimize model
m1.optimize()

print('Obj: %g' % m1.objVal)

m1.Params.LogFile = ""